In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np
import torch.optim as optim

In [2]:
drug_interaction=pd.read_csv("drug_interaction_drugcomb.csv",low_memory=False)
drug_feature=pd.read_csv("drug_feature.csv",index_col=0)
cell_feature=pd.read_csv("final_cell_line_feature.csv",index_col=0)

In [3]:
drug_interaction.head()

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
0,1,5-FU,ABT-888,A2058,uM,uM,30.869,3.865915,6.256584,-2.951386,5.536903,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.839
1,2,5-FU,ABT-888,A2058,uM,uM,27.460,8.247403,12.333896,3.125927,11.614215,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.430
2,3,5-FU,ABT-888,A2058,uM,uM,29.901,6.063440,11.660209,2.452239,10.940528,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.871
3,4,5-FU,ABT-888,A2058,uM,uM,24.016,-4.280231,5.145209,-4.062761,4.425528,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.986
4,5,5-FU,AZD1775,A2058,uM,uM,66.847,12.284698,15.765467,10.409407,18.656340,5.126836,0.266027,11.471,25.164,76.501,57.193,30.212


In [4]:
drug_interaction.head()

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
0,1,5-FU,ABT-888,A2058,uM,uM,30.869,3.865915,6.256584,-2.951386,5.536903,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.839
1,2,5-FU,ABT-888,A2058,uM,uM,27.460,8.247403,12.333896,3.125927,11.614215,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.430
2,3,5-FU,ABT-888,A2058,uM,uM,29.901,6.063440,11.660209,2.452239,10.940528,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.871
3,4,5-FU,ABT-888,A2058,uM,uM,24.016,-4.280231,5.145209,-4.062761,4.425528,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.986
4,5,5-FU,AZD1775,A2058,uM,uM,66.847,12.284698,15.765467,10.409407,18.656340,5.126836,0.266027,11.471,25.164,76.501,57.193,30.212


In [5]:
drug_interaction.shape

(1048575, 18)

In [6]:
drug_interaction['synergistic_status'] = drug_interaction['synergy_zip'].apply(lambda x: 1 if x > 0 else 0)

In [7]:
drug_interaction=drug_interaction[['drug_row','drug_col','cell_line_name','synergistic_status']]

In [8]:
drug_interaction.head()

,drug_row,drug_col,cell_line_name,synergistic_status
0,5-FU,ABT-888,A2058,1
1,5-FU,ABT-888,A2058,1
2,5-FU,ABT-888,A2058,1
3,5-FU,ABT-888,A2058,0
4,5-FU,AZD1775,A2058,1


In [9]:
drug_interaction.isnull().sum()

drug_row                   0
drug_col              582542
cell_line_name             0
synergistic_status         0
dtype: int64

In [10]:
drug_interaction.dropna(subset=['drug_col'], inplace=True)

In [11]:
drug_interaction.isnull().sum()

drug_row              0
drug_col              0
cell_line_name        0
synergistic_status    0
dtype: int64

In [12]:
drug_interaction.synergistic_status.value_counts()

synergistic_status
0    258510
1    207523
Name: count, dtype: int64

In [13]:
interaction_drug= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [14]:
len(interaction_drug)

4150

In [15]:
drug_feature.head()

,drug,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5-FU,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABT-888,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AZD1775,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BEZ-235,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BORTEZOMIB,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
all_drug_with_feature=set(drug_feature['drug'].unique())

In [17]:
len(all_drug_with_feature)

4051

##### if all interaction drug has feature or not

In [18]:
if interaction_drug.issubset(all_drug_with_feature):
    print("All interaction drugs are present in the feature set.")
else:
    print(" Some interaction drugs are missing in the feature set.")
    missing = interaction_drug - all_drug_with_feature
    print("Missing drugs:", missing)


 Some interaction drugs are missing in the feature set.
Missing drugs: {'Ergosterine', 'KS-00001CUZ', 'ZINC11616261', 'AC1NUNML', 'ZINC34894448', 'ZINC117147304', 'MOLPORT-042-665-727', 'AC1L9B3J', 'CTK8E8456', 'ERK 11e', 'AC1L34YQ', '1beta-Methylimipenem', 'AC1L1KJK', 'AC1O7H3F', 'AC1MWLHC', 'ZINC3881972', 'ZINC101426918', 'Curcumin, Curcuma longa L.', 'ZINC100015731', 'KS-00001D5I', 'AC1O53DL', 'ZINC84688828', '04/04/7218', 'AC1L9B2S', 'Vincristine Sulfate, Apocynaceae sp.', 'AC1Q29DP', 'ZINC139045355', 'AC1OF4NF', 'C10H10O4', 'AK160201', 'AK170455', 'KGBPLKOPSFDBOX-UHFFFAOYSA-N', "3',4'-Diacetylafzelin", 'DSSTox_CID_28582', 'PIFITHRIN-A (PFTA)', 'PLURISIN #1 (NSC 14613)', 'ZINC101112865', 'ZINC103939005', 'Pd-1/pd-l1 inhibitor 1', 'ZINC18182079', 'AC1OFCD4', 'AC1O7GND', 'GYNOSTEMMA EXTRACT', 'AC1LCZJT', 'AC1OF33F', 'AK-77283', 'ZINC28538988', 'AC1L21NN', '(6-)E-<U+200B>AMINOCAPROIC ACID', 'ZINC208949883', 'AC1LAKOZ', 'ONO-4059 analog', 'CTK8F0346', 'ZINC3831133', 'AC1LAYFC', 'ZINC43

In [19]:
drug_interaction = drug_interaction[
    ~drug_interaction['drug_row'].isin(missing) &
    ~drug_interaction['drug_col'].isin(missing)
].reset_index(drop=True)

In [20]:
drug_interaction.shape

(464744, 4)

In [21]:
all_drugs_before = set(drug_interaction['drug_row']) | set(drug_interaction['drug_col'])
all_drugs_after = set(drug_interaction['drug_row']) | set(drug_interaction['drug_col'])

removed_drugs = all_drugs_before - all_drugs_after
print(f"Number of unique drugs before: {len(all_drugs_before)}")
print(f"Number of unique drugs after: {len(all_drugs_after)}")
print(f"Drugs removed: {len(removed_drugs)}")
print(f"Drugs removed list: {removed_drugs}")

Number of unique drugs before: 4051
Number of unique drugs after: 4051
Drugs removed: 0
Drugs removed list: set()


In [22]:
interaction_drug= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [23]:
interaction_drug_new= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [24]:
len(interaction_drug),len(interaction_drug_new)

(4051, 4051)

In [25]:
all_drug_with_feature==interaction_drug_new

True

In [26]:
drug_feature['drug_id'] = range(len(drug_feature))

In [27]:
drug_to_id = dict(zip(drug_feature['drug'], drug_feature['drug_id']))

In [28]:
drug_feature.drop(columns=['drug_id','drug'],inplace=True)

In [29]:
drug_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
drug_feature.shape

(4051, 2048)

In [31]:
drug_interaction['drug_row_id'] = drug_interaction['drug_row'].map(drug_to_id)
drug_interaction['drug_col_id'] = drug_interaction['drug_col'].map(drug_to_id)

In [32]:
drug_interaction = drug_interaction.drop(columns=['drug_row', 'drug_col'])

In [33]:
cols = ['drug_row_id', 'drug_col_id'] + [col for col in drug_interaction.columns if col not in ['drug_row_id', 'drug_col_id']]
drug_interaction = drug_interaction[cols]

In [34]:
drug_interaction.head()

,drug_row_id,drug_col_id,cell_line_name,synergistic_status
0,0,1,A2058,1
1,0,1,A2058,1
2,0,1,A2058,1
3,0,1,A2058,0
4,0,2,A2058,1


In [35]:
drug_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
cos_sim_matrix = cosine_similarity(drug_feature)

In [37]:
cos_sim_matrix

array([[1.        , 0.17699808, 0.11211037, ..., 0.11646819, 0.09727208,
        0.14808722],
       [0.17699808, 1.        , 0.2073627 , ..., 0.28200837, 0.22898571,
        0.21912525],
       [0.11211037, 0.2073627 , 1.        , ..., 0.17366199, 0.20719896,
        0.34698416],
       ...,
       [0.11646819, 0.28200837, 0.17366199, ..., 1.        , 0.21525296,
        0.27526932],
       [0.09727208, 0.22898571, 0.20719896, ..., 0.21525296, 1.        ,
        0.26000576],
       [0.14808722, 0.21912525, 0.34698416, ..., 0.27526932, 0.26000576,
        1.        ]], shape=(4051, 4051))

In [38]:
cos_sim_matrix.shape

(4051, 4051)

In [39]:
interaction_drug= set(drug_interaction['drug_row_id']).union(set(drug_interaction['drug_col_id']))

In [40]:
len(interaction_drug)

4051

In [41]:
drug_feature.shape

(4051, 2048)

In [42]:
df_synergy = drug_interaction[drug_interaction['synergistic_status'] == 1]


In [43]:
df_synergy

,drug_row_id,drug_col_id,cell_line_name,synergistic_status
0,0,1,A2058,1
1,0,1,A2058,1
2,0,1,A2058,1
4,0,2,A2058,1
5,0,2,A2058,1
...,...,...,...,...
464739,4046,1823,KB-ChR-8-5-11,1
464740,4047,1823,KB-ChR-8-5-11,1
464741,4048,1823,KB-ChR-8-5-11,1
464742,4049,1823,KB-ChR-8-5-11,1


In [44]:
num_drugs = max(drug_interaction['drug_row_id'].max(), drug_interaction['drug_col_id'].max()) + 1


In [45]:
num_drugs

np.int64(4051)

In [46]:
adj_matrix = np.zeros((num_drugs, num_drugs))


In [47]:
adj_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4051, 4051))

In [48]:
for _, row in df_synergy.iterrows():
    i = int(row['drug_row_id'])
    j = int(row['drug_col_id'])
    adj_matrix[i, j] = 1
    adj_matrix[j, i] = 1  

In [49]:
np.fill_diagonal(adj_matrix, 0)

In [50]:
adj_matrix.shape

(4051, 4051)

In [51]:
adj_matrix

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(4051, 4051))

In [52]:
cos_sim_matrix

array([[1.        , 0.17699808, 0.11211037, ..., 0.11646819, 0.09727208,
        0.14808722],
       [0.17699808, 1.        , 0.2073627 , ..., 0.28200837, 0.22898571,
        0.21912525],
       [0.11211037, 0.2073627 , 1.        , ..., 0.17366199, 0.20719896,
        0.34698416],
       ...,
       [0.11646819, 0.28200837, 0.17366199, ..., 1.        , 0.21525296,
        0.27526932],
       [0.09727208, 0.22898571, 0.20719896, ..., 0.21525296, 1.        ,
        0.26000576],
       [0.14808722, 0.21912525, 0.34698416, ..., 0.27526932, 0.26000576,
        1.        ]], shape=(4051, 4051))

In [53]:
drug_feature=np.array(drug_feature)

In [54]:
X = torch.tensor(drug_feature, dtype=torch.float32)        
A_label = torch.tensor(adj_matrix, dtype=torch.float32)
A_cos = torch.tensor(cos_sim_matrix, dtype=torch.float32)

In [55]:
cosine_label = np.load('cosine.npy')
adj_label = np.load('adj_label.npy')

In [60]:
cosine_label = np.load('cosine.npy')
encoded_np=np.load("cell_feature.npy")

In [61]:
# encoded_np = encoded_features.cpu().numpy()  # if on GPU, else just .numpy()

# Extract cell line names (assuming your original df is called cell_feature)
cell_names = cell_feature.iloc[:, 0].reset_index(drop=True)

# Create new DataFrame
encoded_df = pd.DataFrame(encoded_np, columns=[f'encoded_feat_{i}' for i in range(encoded_np.shape[1])])

# Combine cell names with encoded features
final_df = pd.concat([cell_names, encoded_df], axis=1)

In [62]:
final_df

,cell_line_name,encoded_feat_0,encoded_feat_1,encoded_feat_2,encoded_feat_3,encoded_feat_4,encoded_feat_5,encoded_feat_6,encoded_feat_7,encoded_feat_8,...,encoded_feat_118,encoded_feat_119,encoded_feat_120,encoded_feat_121,encoded_feat_122,encoded_feat_123,encoded_feat_124,encoded_feat_125,encoded_feat_126,encoded_feat_127
0,A2058,15.151784,7.775493,10.975723,-5.267719,1.753620,-64.393517,15.232997,-12.658575,-41.654945,...,-4.550274,-1.442379,11.892857,16.004200,-28.387133,-4.289468,11.827756,-16.554157,-17.257122,2.971671
1,A2780,-28.791862,32.370430,-10.241742,3.092080,-0.712900,-94.673988,2.950004,9.490631,-2.200254,...,-15.053715,-4.154295,-2.901681,3.218300,2.459517,24.622250,-1.195875,-10.838488,100.351799,-24.112007
2,A427,-27.698307,24.713692,-4.823103,-0.788604,4.545676,-70.551956,5.474706,-7.061625,-5.571362,...,-29.496468,-15.435703,0.966603,24.906418,0.942463,-5.809993,36.073700,24.482231,6.445782,-53.730080
3,NCI-H460,16.857882,47.426685,-3.873967,2.696507,8.355028,-60.474964,22.467136,-1.508828,-51.894943,...,-15.911158,-10.053715,2.602337,-22.269821,-9.147532,14.729938,11.208708,11.145536,3.174129,-35.322334
4,RKO,22.336006,30.052328,12.942274,7.180133,2.478162,-46.920300,-1.604194,18.245089,-23.219601,...,-18.677170,-7.680096,6.811165,-44.112957,-13.342626,37.429226,-33.393600,5.212832,37.711433,-31.169086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,SNU-503,-23.421787,-1.154297,3.465809,-7.524144,12.357769,-8.794785,-21.204453,20.703882,-12.212008,...,-8.153191,-2.701307,10.710336,-20.250763,-8.404342,15.764380,-14.648253,-10.452042,144.511200,17.771906
713,SNU-869,66.197929,41.852665,-34.854660,11.098589,3.043755,13.735495,-1.491556,-0.508745,-2.815784,...,-53.961594,4.263747,-4.502186,-46.650307,2.657958,8.374094,-18.806238,-92.930222,65.556816,-23.835127
714,UM-UC-1,72.480194,55.865860,8.613798,-2.045092,4.732880,-22.308880,-25.648197,-16.524408,13.502949,...,1.102995,0.906395,-9.109099,-53.263210,5.469553,-45.714622,-19.944328,-15.440395,24.469955,-25.878483
715,HuG1-N,83.811386,58.386013,4.762500,-22.323183,2.139283,-96.119492,-3.672203,-52.585327,-66.355873,...,57.807301,2.179843,26.427444,2.553859,-52.222557,-43.460186,20.283966,30.906605,33.436081,11.677414


In [107]:
final_df.to_csv("final_ifneeded_Cell.csv")

In [58]:
final_df

,cell_line_name,encoded_feat_0,encoded_feat_1,encoded_feat_2,encoded_feat_3,encoded_feat_4,encoded_feat_5,encoded_feat_6,encoded_feat_7,encoded_feat_8,...,encoded_feat_118,encoded_feat_119,encoded_feat_120,encoded_feat_121,encoded_feat_122,encoded_feat_123,encoded_feat_124,encoded_feat_125,encoded_feat_126,encoded_feat_127
0,A2058,-0.011479,-0.041577,-0.014936,-0.017998,-0.017056,-0.018718,-0.019897,-0.017022,-0.019518,...,-0.035203,-0.032796,-0.023598,-0.025250,-0.019950,-0.016015,-0.072013,-0.034945,-0.018412,-0.012245
1,A2780,-0.009396,-0.031465,-0.011224,-0.012710,-0.013231,-0.010890,-0.014250,-0.011890,-0.014203,...,-0.019873,-0.025375,-0.019546,-0.018911,-0.013469,-0.013256,-0.043791,-0.018959,-0.013370,-0.010332
2,A427,-0.014417,-0.048918,-0.017645,-0.019093,-0.019334,-0.019646,-0.022494,-0.017682,-0.021034,...,-0.034913,-0.039925,-0.035198,-0.030548,-0.021248,-0.018766,-0.108867,-0.019462,-0.022848,-0.013991
3,NCI-H460,-0.014510,-0.046097,-0.016940,-0.021545,-0.019146,-0.019504,-0.018790,-0.018907,-0.020869,...,-0.035260,-0.040036,-0.030710,-0.029588,-0.019608,-0.019432,-0.069339,-0.002327,-0.025819,-0.015341
4,RKO,-0.015343,-0.053351,-0.019520,-0.024078,-0.021794,-0.024493,-0.025836,-0.022683,-0.025124,...,-0.047460,-0.043335,-0.033032,-0.033497,-0.028199,-0.021099,-0.096338,-0.030114,-0.024940,-0.016597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4046,NaN,-0.009842,-0.025942,-0.008001,-0.010043,-0.009440,-0.012576,-0.005741,-0.008927,-0.006908,...,-0.012366,-0.025894,-0.019270,-0.019849,0.632720,-0.009006,-0.030844,1.785376,-0.011025,-0.008136
4047,NaN,-0.008155,-0.022243,-0.008238,-0.009189,-0.009826,-0.009294,-0.009543,-0.007829,-0.010352,...,-0.011146,-0.019169,-0.021060,-0.016121,-0.012237,-0.008953,-0.032241,3.033585,-0.009341,-0.008670
4048,NaN,-0.006011,-0.026847,-0.007668,-0.001494,0.236853,-0.010748,-0.029939,0.099770,-0.005079,...,-0.024184,-0.027594,-0.023145,-0.027334,-0.017378,-0.001797,-0.139669,-0.092446,-0.002718,-0.002369
4049,NaN,0.278728,-0.024805,-0.008621,-0.005828,-0.009190,-0.010585,-0.016836,-0.004017,-0.007466,...,-0.031537,-0.028579,-0.025901,-0.027793,-0.016087,-0.000083,-0.146886,-0.052834,-0.003430,-0.000864


In [63]:
import numpy as np
import pandas as pd



cell_feat_map = final_df.set_index(final_df.columns[0])  # index on cell_line_name, columns are features

# Prepare lists to collect features
combined_features = []
labels = []
skipped_rows = 0

for idx, row in drug_interaction.iterrows():
    drug_row_idx = row['drug_row_id']
    drug_col_idx = row['drug_col_id']
    cell_name = row['cell_line_name']
    label = row['synergistic_status']

    # Check if cell name exists in final_df
    if cell_name not in cell_feat_map.index:
        skipped_rows += 1
        continue  # skip this row

    # Extract drug features
    try:
        drug_row_feat = cosine_label[drug_row_idx]
        drug_col_feat = cosine_label[drug_col_idx]
    except IndexError:
        skipped_rows += 1
        continue  # skip if drug id is out of bounds

    # Extract cell features
    cell_feat = cell_feat_map.loc[cell_name].values  # shape (128,)

    # Combine all
    combined = np.concatenate([drug_row_feat, drug_col_feat, cell_feat])  # (384,)
    combined_features.append(combined)
    labels.append(label)

# Convert to final DataFrame
combined_features = np.array(combined_features)
labels = np.array(labels)

feature_cols = [f'drug_row_feat_{i}' for i in range(128)] + \
               [f'drug_col_feat_{i}' for i in range(128)] + \
               [f'cell_feat_{i}' for i in range(128)]

final_dataset = pd.DataFrame(combined_features, columns=feature_cols)
final_dataset['synergistic_status'] = labels

print(f"Final dataset shape: {final_dataset.shape}")
print(f"Skipped rows due to missing cell line or drug index: {skipped_rows}")
print(final_dataset.head())

Final dataset shape: (166227, 385)
Skipped rows due to missing cell line or drug index: 298517
   drug_row_feat_0  drug_row_feat_1  drug_row_feat_2  drug_row_feat_3  \
0        -0.008872        -0.003799        -0.011806        -0.002723   
1        -0.008872        -0.003799        -0.011806        -0.002723   
2        -0.008872        -0.003799        -0.011806        -0.002723   
3        -0.008872        -0.003799        -0.011806        -0.002723   
4        -0.008872        -0.003799        -0.011806        -0.002723   

   drug_row_feat_4  drug_row_feat_5  drug_row_feat_6  drug_row_feat_7  \
0        -0.015133         0.013551        -0.010493         -0.00656   
1        -0.015133         0.013551        -0.010493         -0.00656   
2        -0.015133         0.013551        -0.010493         -0.00656   
3        -0.015133         0.013551        -0.010493         -0.00656   
4        -0.015133         0.013551        -0.010493         -0.00656   

   drug_row_feat_8  drug_ro

np.int64(0)

In [109]:
final_dataset.to_csv("fianl_interaction.csv")

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
X = final_dataset.iloc[:, :-1].values  # all features
y = final_dataset.iloc[:, -1].values

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [69]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
model = LogisticRegression(max_iter=1000)  # Increase max_iter if convergence warning
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.6289177645431029
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.87      0.73     19236
           1       0.63      0.29      0.40     14010

    accuracy                           0.63     33246
   macro avg       0.63      0.58      0.57     33246
weighted avg       0.63      0.63      0.59     33246

Confusion Matrix:
 [[16813  2423]
 [ 9914  4096]]


C:\Users\lenovo\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
